In [ ]:
from pyspark import SparkContext
from awsglue.context import GlueContext
import  boto3

In [ ]:
import sys
print(sys.version)

In [ ]:
s3Client = boto3.client('s3')
s3Bucket="dataplatform-stg-landing-zone"
prefix = "parking/manual"

objects = s3Client.list_objects_v2(
    Bucket=s3Bucket,
    Prefix=prefix
)['Contents']

nonEmptyObjects = list(filter(lambda x: x['Size'] != 0, objects))
nonEmptyObjects[0:10]

In [ ]:
exampleObject = nonEmptyObjects[0]
glueContext = GlueContext(SparkContext.getOrCreate())
inputDF = glueContext.create_dynamic_frame_from_options(
    connection_type = "s3",
    connection_options = {"paths": [f"s3://{s3Bucket}/{exampleObject}"]},
    format = "csv",
    format_options = { "withHeader": True }
)

inputDF.toDF().show(10)
inputDF.toDF().columns